### Base Learner 7f  - Load Fit10 Model, fit_once_cylce slice on top

In [7]:
from fastai2.vision.all import *
import sys, torch
sys.path.append('..')

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


# %load_ext autoreload
# %autoreload 2
from modules.learnutils import TestSetRecorder
from modules.expbuilder import default_params, run_exp, rm_test_recorder

train_path = Path('../../../rf-chess-data/cropped_v1/')
train_clean_path = Path('../../../rf-chess-data/cropped_v1.1/')

test_path_all = Path('../../../other-chess-data/regulation-test-2-all/')
test_path_easy = Path('../../../other-chess-data/regulation-test-2-easy/')

In [8]:
learn = load_learner('../models/base-learner7-fit10.pkl')
learn.path = Path('../../../rf-chess-data/cropped_v1.1')

my_params = {'_train_seed':0, '_custom_crop': 'my-top-crop', 
             '_model_arch': resnet50,
            '_train_path':train_clean_path,
            '_learn_norm': False,
            }
default_params.update(my_params)
set_seed(42)

train_dl, learn_fresh = run_exp(default_params, 'dummy', b_ret=True,)

learn.cbs.pop(3)

learn.dls = learn_fresh.dls

learn.add_cb(TestSetRecorder())

##### New Fit

In [3]:
learn.unfreeze()

In [4]:
set_seed(42)
learn.fit_one_cycle(9, lr_max=slice(1e-6,1e-4))

epoch,train_loss,valid_loss,accuracy,subcat_color_acc,subcat_piece_acc,time
0,0.120849,0.001298,1.000000,1.000000,1.000000,00:14
1,0.073554,0.002223,0.998258,0.998258,0.998258,00:14
2,0.047045,0.001834,1.000000,1.000000,1.000000,00:14
3,0.025016,0.002325,0.998258,0.998258,0.998258,00:14
4,0.015420,0.003089,0.998258,0.998258,0.998258,00:14
5,0.011352,0.002442,0.998258,0.998258,0.998258,00:14
6,0.010414,0.002119,0.998258,0.998258,0.998258,00:14
7,0.015071,0.002739,0.998258,0.998258,0.998258,00:14
8,0.039555,0.002045,1.000000,1.000000,1.000000,00:14


In [10]:
pd.DataFrame(learn.cbs[3].values)

,0,1,2,3
0,0.846266,0.741228,0.969298,0.789474


In [9]:
easy_test_dl = learn.dls.test_dl(get_image_files(test_path_easy), 
                                 with_labels=True)

interp = ClassificationInterpretation.from_learner(learn, 
                                                  dl = easy_test_dl)

interp.print_classification_report()

              precision    recall  f1-score   support

black-bishop       0.86      0.67      0.75        18
  black-king       0.80      0.84      0.82        19
black-knight       0.60      0.95      0.73        19
  black-pawn       0.76      1.00      0.86        19
 black-queen       1.00      0.25      0.40        20
  black-rook       0.68      0.89      0.77        19
white-bishop       0.57      0.68      0.62        19
  white-king       0.72      0.68      0.70        19
white-knight       0.84      0.84      0.84        19
  white-pawn       0.85      0.89      0.87        19
 white-queen       1.00      0.32      0.48        19
  white-rook       0.74      0.89      0.81        19

    accuracy                           0.74       228
   macro avg       0.78      0.74      0.72       228
weighted avg       0.79      0.74      0.72       228

